In [1]:
%load_ext autoreload
%autoreload 2
import cnn
import xarray as xr
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import sklearn.metrics as skl
device = torch.device('cuda:0')

In [2]:
device

device(type='cuda', index=0)

In [3]:
from sklearn.utils import shuffle
from tqdm import tqdm
from IPython.display import clear_output


In [4]:
# ds_first = xr.open_mfdataset('/home/ronja/Praktikum-IFU/example_data.nc', engine='netcdf4')
ds = xr.open_dataset('/bg/fast/aihydromet/cml_wet_dry_radklim/train_data_combined.nc', engine='netcdf4',)
ds = ds.load()
ds.nbytes/1e9

In [33]:
# ds_first = xr.open_mfdataset('/home/ronja/Praktikum-IFU/example_data.nc', engine='netcdf4')
ds_test = xr.open_dataset('/bg/fast/aihydromet/cml_wet_dry_radklim/train_data_combined_06.nc', engine='netcdf4',)
ds_test = ds_test.load()
ds_test.nbytes/1e9

27.595304412

In [34]:
batchsize = 1000
train_size = int(len(ds.sample_number)*0.8/batchsize)* batchsize
test_size = int(len(ds.sample_number)*0.2/batchsize)* batchsize

In [8]:
working_size

9914000

In [35]:
tl, wr = shuffle(ds.tl.values, ds.wet_radar.values)
tl_test, wr_test = shuffle(ds_test.tl.values, ds_test.wet_radar.values)


In [ ]:

data = tl[:train_size]
data_out = wr[:train_size]


In [37]:
data_test = tl_test[:test_size]
data_test_out = wr_test[:test_size]

In [38]:
np.histogram(data_test_out)

/tmp/ipykernel_44111/2312033233.py:1: RuntimeWarning: Converting input from bool to <class 'numpy.uint8'> for compatibility.
  np.histogram(data_test_out)


(array([1239895,       0,       0,       0,       0,       0,       0,
              0,       0, 1238105]),
 array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]))

bsp zum "abrunden" auf ganze batch size
`data = data[:int(len(data/batch_size))*batch_size]`

In [39]:

### data as Tensor
data = torch.Tensor(data).type(torch.FloatTensor).to(device)
data_test = torch.Tensor(data_test).type(torch.FloatTensor).to(device)

data_out = torch.Tensor(data_out).type(torch.FloatTensor).to(device)
data_test_out = torch.Tensor(data_test_out).type(torch.FloatTensor).to(device)


In [40]:

### prepare data
dataset = torch.utils.data.TensorDataset(data, data_out)
testset = torch.utils.data.TensorDataset(data_test, data_test_out)
trainloader = torch.utils.data.DataLoader(dataset, batch_size = batchsize, shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = batchsize, shuffle = False)


In [41]:
model = cnn.cnn_class()
model = model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001, amsgrad=True)


In [42]:
next(model.parameters()).is_cuda

True

In [43]:
epochs = 10
resume = 7

In [44]:
if resume == 0:
    loss_dict = {}
    loss_dict['train'] = {}
    loss_dict['test'] = {}
    for key in ['train','test']:
        loss_dict[key]['loss'] = []

In [45]:
for epoch in range(resume, epochs):
    # training
    train_losses = []
    for inputs, targets in tqdm(trainloader):
        optimizer.zero_grad()
        pred = model(inputs)
        pred = nn.Flatten(0,1)(pred)
        loss = nn.BCELoss()(pred, targets)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.detach().cpu().numpy())
    loss_dict['train']['loss'].append(np.mean(train_losses))
    
    # testing
    test_losses = []
    with torch.no_grad():
        for inputs, targets in tqdm(testloader):
            pred = model(inputs)
            # pred = pred.round()
            pred = nn.Flatten(0,1)(pred)
            loss = nn.BCELoss()(pred, targets)
            test_losses.append(loss.detach().cpu().numpy())
        loss_dict['test']['loss'].append(np.mean(test_losses))
        
    # printing
    clear_output(wait=True)
    print(epoch)
    print('')
    print('train loss:', np.mean(train_losses))
    print('test loss:', np.mean(test_losses))
    print('min test loss:', np.min(loss_dict['test']['loss']))
    fig, axs = plt.subplots(1,1, figsize=(3,4))
    for key in loss_dict.keys():
        for k, key2 in enumerate(loss_dict[key].keys()):
            axs.plot(loss_dict[key][key2], label=key)
            axs.set_title(key2)
    # axs.set_yscale('log')
    plt.legend()
    plt.show()
    resume = epoch

 15%|█▌        | 1531/9914 [00:41<03:31, 39.70it/s]

In [30]:
### accuracy score, confusion matrix and Matthews correlation coefficient (MCC)
def eval_model(test_dataloader, model):
    predictions, actuals = list(), list()
    with torch.no_grad():
        for inputs, targets in tqdm(test_dataloader):
            pred = model(inputs)
            pred = pred.detach().cpu().numpy()
            actual = targets.cpu().numpy()
            actual = actual.reshape((len(actual), 1))
            pred = pred.round()
    
            predictions.append(pred)
            actuals.append(actual)
    predictions, actuals = np.ravel(np.vstack(predictions)), np.ravel(np.vstack(actuals))
    acc = skl.accuracy_score(actuals, predictions)
    conf_mat = skl.confusion_matrix(actual, pred, labels=[0,1], normalize='true')
    matt_cor = skl.matthews_corrcoef(actual, pred)
    return acc, conf_mat, matt_cor

In [31]:
eval_model(testloader, model)

  0%|          | 0/2480 [00:00<?, ?it/s]

100%|██████████| 2480/2480 [00:19<00:00, 129.25it/s]


(0.9347308103678299,
 array([[0.9034749 , 0.0965251 ],
        [0.06225681, 0.93774319]]),
 0.8416164715349778)